In [81]:
from pymarc import Field, MARC8ToUnicode, Record, marcxml, XMLWriter
import json
import datetime
dt_now = datetime.datetime.now().strftime("%Y%m%d")
from collections import defaultdict
import pandasgui
from copy import deepcopy
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive
from VC_collections.AuthorityFiles import Authority_instance
from VC_collections.authorities import check_lang, find_name, find_role, map_relators
from VC_collections.AuthorityFiles import create_df_from_gs
import re

import janitor

## TOC:
* [Get input file of BS records from Beit Ariela](#1)
* [Set up output files](#2)
* [Working with test record](#3)
    * [Which fields to keep](#3.1)
* [Checking fields](#4)
    * [Checking 245](#4.1)
        * [245h](#4.1.1)
        * [080](#4.1.2)
* [Set up output files](#2)
* [Set up output files](#2)

# Get input file of BS records from Beit Ariela <a class="anchor" id="1"></a>

In [3]:
batsheva_records = marcxml.parse_xml_to_array("./Data/BatSheva/batsheva1907.xml")

In [4]:
# how many records are in the xml export?
len(batsheva_records)

1511

# Set up output files <a class="anchor" id="2"></a>

brief records will contain the following data: 
- LDR
- 008
- 035
- 093 
- 245
- 650

In [5]:
brief_BS_records = r"G:\My Drive\National_Library\Python\VC-Dance\DCBS\Data\processed\DCBS_brief.xml"

# Working with test record <a class="anchor" id="3"></a>

In [6]:
test_record = json.loads((batsheva_records[0].as_json()))
type(batsheva_records[20])

pymarc.record.Record

In [7]:
print(batsheva_records[1000])

=LDR  00000nkm  22      a 4500
=001  000507431
=005  20190401195224.0
=008  061102s1995\\\\is\nnn\\\\\\\\\\\\i|heb\d
=041  \\$aheb
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=110  2\$aלהקת מחול בת שבע$9heb
=245  10$aזינה$h [תמונה]/$cצילום: גדי דגון.
=260  \\$aישראל :$bדגון, גדי,$c1995.
=300  \\$a503 תצלומים :$bשחור/לבן & צבע.
=500  \\$aבכורה עולמית – 20 למאי 1995 - תאטרון ירושלים, ירושלים. נוצר לערב הפתיחה של פסטיבל ישראל 1995, ירושלים. מנהל אמנותי פסטיבל ישראל: מיכה לבינסון
=508  \\$aChoreography: Ohad Naharin
=508  \\$aכוריאוגרפיה: אוהד נהרין
=508  \\$aמוסיקה: עברי לידר, אוהד נהרין. קלידים: עברי לידר. גיטרה: עמית גולדנברג. גיטרה בס: גולן זוסקוביץ. תופים: טל עמירן גיטרה, שירה: אוהד נהרין
=508  \\$aעיצוב סאונד: פרנקי ליוורט
=508  \\$aפסקול כלל הקלטות של: Ice Cube, Ice Tea, Stanely Adams and Maria Grever
=508  \\$aתפאורה: אריק לוי
=508  \\$aתלבושות: רקפת לוי
=508  \\$aתאורה: במבי
=508  \\$aMusic: Ivry Lider, Ohad Naharin. Keyboard: Ivry Lider. Guitar: Amit Goldenberg. Bass Guit

In [337]:
def get_record_by_sysid(batsheva_records, sys_id):
    counter = 0
    for record in batsheva_records:
        if record.get_fields("001")[0].value() == sys_id:
            print(f'record index: {counter}')
            return record
        counter +=1
    print("record not found")
    return ""
    

In [339]:
print(get_record_by_sysid(batsheva_records, "000665761"))

record index: 1268
=LDR  00000npc a22        4500
=001  000665761
=005  20180411152403.0
=008  091203m1981\\\\is\\\\\\|\\\\\\\\\\\heb\d
=041  0\$aheb
=080  \\$aBAL 111.6.5.2.6 הרו נו-אמי חסוי
=092  \\$a[14]
=100  1\$aTillis, Barry$9lat$elighting designer$g- lighting programs
=245  10$aהרו נו-אמי$h [תכנית תאורה]
=260  \\$aישראל :$bלהקת מחול בת-שבע,$c1981.
=540  \\$aזכויות היוצרים ובעלים: להקת בת שבע
=561  \\$aנמסר על ידי הלהקה 2015
=500  \\$aהועלה לראשונה בסטודיו של קאזוקו היראביאשי, ניו יורק, 1980. בכורה בלהקת בת-שבע – 15 אפריל 1981 – אולם נחמני, תל-אביב. הועלה 45 פעמים
=508  \\$aכוריאוגרפיה: אוהד נהרין
=508  \\$aמוסיקה: מיצ'יו מייאגי בביצוע של ז'אן פייר רמפל
=508  \\$aתאורה: ברי טיליס
=508  \\$aChoreography: Ohad Naharin
=508  \\$aMusic: Michio Miyagi by Jean-Pierre Rampal
=508  \\$aLighting: Barry Tillis
=505  8\$aתכנית תאורה.
=630  04$aהרו נו-אמי (כור': נהרין, אוהד. 1981)$vתכניות תאורה$x(מחול)
=630  04$aHaru No Umi (Choreographic Work: Naharin, Ohad. 1981)$vlighting programs$x(Dance

## Which fields to keep: <a class="anchor" id="3.1"></a>
- **001** = __not needed__ - new MMS ids for Alme will be created.
- **035** = Control number for the record in a system other than the one whose control number is contained in field 001.  the format is #######SZB. (Check if exist in all records)
- **041** - Keep exact.
- **080** - Universal Decimal Classification Number - currently holding place of digital files. __ask Viki_
- **092** - Local Call Numbers. __not needed__
- **245** - keep exact
- **260** - keep exact
- **300** - keep exact
- **500** - keep exact
- 

1. create an XMLwriter object for writing into the output file
2. Creating a new empty record and filling it with the fields from the BS records Beit Ariela export

In [264]:
writer = XMLWriter(open(brief_BS_records, "wb"))
new_test_record = Record()

# Checking fields <a class="anchor" id="4"></a>

number of records

In [265]:
len(batsheva_records)

1511

In [266]:
type(batsheva_records)

list

In [267]:
def check_field_occurances(records:list, tag:str, subfield=""):
    """
    This function takes a list of pymarc records and checks how many occurances a given tag has
    """
    count = 0
    for record in records:
        if subfield == "":
            if len(record.get_fields(tag)) > 0:
                count+=1
        else:
            if len(record.get_fields(tag)[0][subfield]) > 0:
                count += 1
                
    print(f'{tag}{subfield} field appears in {count} out of {len(records)}')

## Checking 245 <a class="anchor" id="4.1"></a>

number of 245

In [268]:
check_field_occurances(batsheva_records, tag = "245")

245 field appears in 1511 out of 1511


### 245\$h <a class="anchor" id="4.1.1"></a>

Field 245$h contains the material type. This field can be used to create the following fields: 
- 655
- 999

In [269]:
check_field_occurances(batsheva_records, tag="245", subfield="h")

245h field appears in 1511 out of 1511


In [270]:
h_245  = defaultdict(list)
h_245_lengh = dict()


for record in batsheva_records:
    h_245[record.get_fields("245")[0]["h"].strip().strip("/").strip(":").rstrip().strip(".")].append(record.get_fields("001")[0].value())
    
    
h_245_lengh = {i: len(j) for i, j in  h_245.items()}

In [271]:
import pprint
pprint.pprint(sorted(h_245_lengh.items()))

[('[archival material]', 2),
 ('[music]', 1),
 ('[picture]', 2),
 ('[program booklet]', 5),
 ('[videorecording]', 8),
 ('[הקלטת וידאו]', 214),
 ('[חומר ארכיוני]', 590),
 ('[תווים]', 2),
 ('[תיק ארכיון]', 11),
 ('[תיק הפקה]', 1),
 ('[תכניה]', 75),
 ('[תכנית תאורה]', 24),
 ('[תמונה]', 576)]


### 080 <a class="anchor" id="4.1.2"></a>

The call number of BABAL (Beit Ariela) is based on the- [Universal Decimal Classification scheme](https://en.wikipedia.org/wiki/Universal_Decimal_Classification). Or the dewey system

It looks like a UDC notation but it doesn't contain the true classification (for example, Dance should be 793.3 - as seen [here >>](https://udcsummary.info/php/index.php?id=64676&lang=en#) 

The original field should be kept in the import. 
This field will be the base of the IAN call number generation. 

# Copy MARC21 080 Field <a class="anchor" id="5"></a>

In [272]:
def copy_080(input_record, new_record):
    record = deepcopy(input_record)
    
    fields = record.get_fields("080")
    
    for field in fields: 
        new_record.add_ordered_field(field) 
    
    
    return new_record

In [273]:
print(copy_080(batsheva_records[1000], new_test_record))

=LDR            22        4500
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]



In [274]:
dict_080 = defaultdict(list)

for record in batsheva_records:
    fields_080 = record.get_fields("080")[0]["a"]
#     for field in fields_080:
# #         print(field["a"])
    field_001 = record.get_fields("001")[0].value()
    for val in fields_080.split():
        dict_080[field_001].append(val)
    

In [275]:
try:
    df = pd.DataFrame.from_dict(dict_080, orient='index').to_excel("BS_call_numbers.xlsx")
except PermissionError:
    print("file open, please close")
# df = df.transpose()

file open, please close


# Create MARC21 093 Field <a class="anchor" id="5"></a>

In order to transform is to an Archival call number, the following steps need to be taken:
- handling of duplicates(!!!!)
- removal of hebrew characters
- replace `.` with `-`
- replace ` ` with `-`
- replace `BAL` at begining with `IL-BABAL`
- replace `EStorage BS` with `IL-BABAL-111.6-EStorage`

other cases that need to be resolve:
- square brackets - ex. `BAL ‪[P] 111.6.2.4` - are not allowed and need to be replaced. Decision: will be replaced with parentheses.
- parentheses - ex. `BAL 111.6(1990).5` - parentheses are allowed in url, and not need to be replace with encoding (%28, %29).
- in some cases the value in 080 contains the value "חסוי" - it is very important to keep is somewhere.


- 093\$c - will contain the Archival call number
- 093\$d - will contain the name of the Archive (hebrew), in this case "hard coded" - `ארכיון להקת מחול בת שבע`
- 093\$e - will contain the name of the Archive (english), in this case "hard coded" - `Batsheva Dance Company Archive`

In [276]:
hebrew_alphabet = "אבגדהוזחטיכךלמנסעפצקרשתםןףץ"

In [277]:
def remove_hebrew_chars(value):
    new_val = ''
    for char in value:
        if char not in hebrew_alphabet:
            new_val += char
    return new_val

In [278]:
test_080 = "EStorage BS 002B סדנה"
print(remove_hebrew_chars(test_080))

EStorage BS 002B 


In [279]:
def change_square_brackets(value):
    new_value = value.replace("[","(").replace("]",")")
    return new_value

In [280]:
def assign_correct_call_number(value):
    new_value = value.replace("BAL", "IL-BABAL").replace("EStorage BS", "IL-BABAL-111.6-EStorage")
    return new_value

In [281]:
test1_080 = "BAL [Ph] 111.6.1.5 ק"
print(assign_correct_call_number(remove_hebrew_chars(change_square_brackets(test_080))).replace(".", "-").replace(" ", "-").rstrip("-"))

IL-BABAL-111-6-EStorage-002B


In [282]:
def create_093(input_record, new_record):
    record = deepcopy(input_record)
    
    data_for_093 = record.get_fields("080")[0]["a"]
    print(data_for_093)
    
    
    
    new_093_val = assign_correct_call_number(data_for_093)
    new_093_val = re.sub("(-){3,}", "-", data_for_093)
    # remove all alpha characters
#     new_093_val = re.sub("[a-zA-Z]*", "", new_093_val) 
    new_093_val = new_093_val.replace(".", "-").replace(" ", "-").replace("()", "").rstrip("-").strip("_")
    new_093_val = re.sub("(_)*", "", new_093_val) 
    new_093_val = remove_hebrew_chars(new_093_val)
    new_093_val = change_square_brackets(new_093_val)
    print(new_093_val)




    field_093 = Field(
        tag = '093',
    indicators = [' ',' '],
    subfields = [
        'c', new_093_val,
        'd', "ארכיון להקת מחול בת שבע",
        'e', "Batsheva Dance Company Archive",
        
    ])
    
    new_record.add_ordered_field(field_093)
    
    return new_record

In [283]:
print(create_093(batsheva_records[1000], new_test_record))

BAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
BAL-111-6(1990)-2-3-Harvard(D)
=LDR            22        4500
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive



## create MARC21 035 Field <a class="anchor" id="6"></a>
field 035 is based on the system number of the records in Beit Ariela's Library system.
Israel Dance Archive and Library does not have a MARC organization code. 
The ISIL code produces by IAN project for Beit Ariela did not meet their need. 
So they changed it to IL-BABAL - this will also be the base of the MARC organization code I invented.

the code will be:
IL-BABAL

035 field will be created as following:
(IL-BABAL){001 of original records}


In [284]:
def create_035(input_record, new_record):

    record = deepcopy(input_record)
    val_035 = f'(IL-BABAL){record.get_fields("001")[0].value()}'
    
    field = Field(
        tag = '035',
    indicators = [' ',' '],
    subfields = [
        'a', val_035,
    ])
    
    new_record.add_ordered_field(field)
    
    return new_record

In [285]:
print(create_035(batsheva_records[26], new_test_record))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive



# clean and create MARC21 245 Field <a class="anchor" id="7"></a>

In [286]:
h245_to_999 = {
'[archival material]': ["ARCHIVE"],  
 '[music]': ["RECCOMM", "ARCHIVE"], #655 - שמע
 '[picture]': ["PHOTOGRAPH", "ARCHIVE"], #655 - תצלום (תצלומים)
 '[program booklet]': ["ARCHIVE"], # 655 - תכנייה (חולפות)
 '[videorecording]': [ "VIDEO", "ARCHIVE"], # 655 - תמונות נעות
 '[הקלטת וידאו]': [ "VIDEO", "ARCHIVE"], #655 - תמונות נעות
 '[חומר ארכיוני]': ["ARCHIVE"], 
 '[תווים]': ["ARCHIVE"], # 655 - תווים (מסמכים-כללי)
 '[תיק ארכיון]': ["ARCHIVE"],
 '[תיק הפקה]': ["ARCHIVE"],
 '[תכניה]': ["ARCHIVE"],  # 655 - תכנייה (חולפות)
 '[תכנית תאורה]': ["ARCHIVE"], # 655 - מפת תאורה (תכניות ושרטוטים)
 '[תמונה]':["PHOTOGRAPH", "ARCHIVE"], # 655 - תצלום (תצלומים)
}

In [287]:
def create_245(input_record, new_record):
    record = deepcopy(input_record)
    
    field = record.get_fields("245")[0]
    val_h = field["h"].strip().strip("/").strip(":").rstrip().strip(".")
    val_a = field["a"].strip()
    
    
    field = Field(
        tag = '245',
        indicators = ['1','0'],
        subfields = [
        'a', val_a,
        'h', val_h
    ])
    
    new_record.add_ordered_field(field)
    
    for val_999 in h245_to_999[val_h]:
        field_999 = Field(
            tag = "999",
            indicators = [" ", " "],
            subfields = [
                'a', val_999, 
                
            ]
        )
#     print(h245_to_999[val_h], "\n")
    
        new_record.add_ordered_field(field_999)
    
    
    return new_record

In [288]:
print(create_245(batsheva_records[1000], new_test_record))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive
=245  10$aזינה$h[תמונה]
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



# Create MARC21 655 field <a class="anchor" id="8"></a>
655 field is based on the value in field 245$h. 
The following mapping dictionary will be used:

In [289]:
list(Authority_instance.df_arch_mat_auth.columns)

['skosxl:broader@prefLabel',
 'skosxl:prefLabel@lang=heb',
 'MARC21 655 7',
 'ARCHIVAL_MATERIAL_ALT_HEB',
 'מדינת הפרסום/הצילום/היצירה',
 'creator_pers',
 'creator_corp',
 'ARCHIVAL_MATERIAL',
 'skosxl:scopeNote@lang=heb',
 'הערך ב-GETTY',
 'rdacontent 336',
 'English',
 'Getty Id',
 'Source',
 'לגלעד']

In [290]:
h245_to_655 = {

 '[music]':  "שמע",
 '[picture]': "תצלום (תצלומים)",
 '[program booklet]':"תכנייה (חולפות)",
 '[videorecording]': "תמונות נעות",
 '[הקלטת וידאו]': "תמונות נעות",
 '[תווים]': "תווים (מסמכים-כללי)",
  '[תכניה]': "תכנייה (חולפות)",
 '[תכנית תאורה]': "מפת תאורה (תכניות ושרטוטים)",
 '[תמונה]':"תצלום (תצלומים)",
    
}

In [291]:
mapping_655 = Authority_instance.df_arch_mat_auth[["ARCHIVAL_MATERIAL","English", "Getty Id", "Source"]].set_index("ARCHIVAL_MATERIAL")

In [292]:
mapping_655.loc[h245_to_655["[music]"], :]

English                                       Sound recording
Getty Id    http://id.loc.gov/vocabulary/graphicMaterials/...
Source                                                    tgm
Name: שמע, dtype: object

In [293]:
def create_655(input_record, new_record, h245_to_655):
    """The sum of two numbers.
   Here is the rest of the docs.
   """
    record = deepcopy(input_record)
#     new_record = create_245(record, new_record)
    
    val_h = new_record.get_fields("245")[0]["h"]
    
    if mapping_655.loc[h245_to_655[val_h], "Getty Id"] != "":

        field_655 = Field(
            tag = "655", 
            indicators = [" ", "7"],
            subfields = [
                "a", mapping_655.loc[h245_to_655[val_h], "English"],
                "2", mapping_655.loc[h245_to_655[val_h], "Source"],
                "0", mapping_655.loc[h245_to_655[val_h], "Getty Id"],
            ]
        )
        
    else:
        ield_655 = Field(
            tag = "655", 
            indicators = [" ", "7"],
            subfields = [
                "a", mapping_655.loc[h245_to_655[val_h], "English"],
                "2", mapping_655.loc[h245_to_655[val_h], "Source"],
            ]
        )
        

    new_record.add_ordered_field(field_655)


    return new_record

In [294]:
print(create_655(batsheva_records[26], new_test_record, h245_to_655))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive
=245  10$aזינה$h[תמונה]
=655  \7$aphotographs$2aat$0http://vocab.getty.edu/aat/300046300
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



# Copy MARC Field 041

In [297]:
def copy_041(input_record, new_record):
    """The sum of two numbers.
   Here is the rest of the docs.
   """
    record = deepcopy(input_record)
    for field in record.get_fields("041"):
        new_record.add_ordered_field(field)
        
    return new_record
        

In [298]:
print(copy_041(batsheva_records[1000], new_test_record))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=041  \\$aheb
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive
=245  10$aזינה$h[תמונה]
=655  \7$aphotographs$2aat$0http://vocab.getty.edu/aat/300046300
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



# Copy MARC Field 100

In [343]:
def copy_100(input_record, new_record):
    """The sum of two numbers.
   Here is the rest of the docs.
   """
    record = deepcopy(input_record)
    for field in record.get_fields("100"):
        field["e"] = field["e"].rstrip(".")
        new_record.add_ordered_field(field)
        
    return new_record

In [344]:
print(copy_100(batsheva_records[1268], new_test_record))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=041  \\$aheb
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive
=100  1\$aTillis, Barry$9lat$elighting designer$g- lighting programs
=100  1\$aTillis, Barry$9lat$elighting designer$g- lighting programs
=110  2\$aלהקת מחול בת שבע$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=245  10$aזינה$h[תמונה]
=655  \7$aphotographs$2aat$0http://vocab.getty.edu/aat/300046300
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



# Copy MARC Field 110

In [341]:
def copy_110(input_record, new_record):
    """The sum of two numbers.
   Here is the rest of the docs.
   """
    record = deepcopy(input_record)
    for field in record.get_fields("110"):
        if field["a"] == "להקת מחול בת שבע" and field["e"] == None:
            field.add_subfield("e", "(יוצרי הארכיון)", 1)
         
        new_record.add_ordered_field(field)
        
    return new_record

In [342]:
print(copy_110(batsheva_records[1000], new_test_record))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=041  \\$aheb
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive
=100  1\$aTillis, Barry$9lat$elighting designer$g- lighting programs
=110  2\$aלהקת מחול בת שבע$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=245  10$aזינה$h[תמונה]
=655  \7$aphotographs$2aat$0http://vocab.getty.edu/aat/300046300
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



# MARC Field 246

a record may contain more than one 246 field.
some of them are duplicates for 245.
we are not doing that check.

The only modification is the following:
- for 246\$a fields that contain English titles, and the main title contains hebrew, then 246 will be formatted as follow:
    - 246\$i will be added, with the prefix `English title:`
    - the field indicators will change to ['3', ' ']
    
The other 246 fields will be copied as is. 
    

In [352]:
def create_246(input_record, new_record):
    """The sum of two numbers.
   Here is the rest of the docs.
   """
    record = deepcopy(input_record)
    lang_245 = check_lang(record.get_fields("245")[0]["a"])
    for field in record.get_fields("246"):
        if check_lang(field["a"]) == "lat" and check_lang(field["a"]) != lang_245:
            field.add_subfield("i", "English title:", 0)
        new_record.add_ordered_field(field)
        
    return new_record

In [356]:
print(get_record_by_sysid(batsheva_records, "000638665"))

record index: 1091
=LDR  00000ngm  22      i 4500
=001  000638665
=005  20190219105259.0
=007  cd\cza||||||||
=008  150610t19911991is\|||\\\\\\\\\\\\||heb
=041  \\$aheb
=080  \\$aEStorage BS 002B
=092  \\$a[14]
=100  1\$aברוך, רינה$9heb$eכוריאוגרף
=245  10$aאובו המלך$h [הקלטת וידאו]
=246  33$aKing Ubu$h [videorecording]
=256  \\$aקובץ מחשב - AVI
=264  \1$aתל אביב :$bלהקת מחול בת-שבע,$c[1991].
=264  \4$c©1991.
=300  \\$a2 קבצי מחשב AVI (38 + 19 דק'). :$bצבע.
=336  \\$atwo-dimensional moving image$btdi$2rdacontent
=337  \\$avideo$bv$2rdamedia
=338  \\$acomputer file$bd$2rdacarrier
=344  \\$adigital$2rda
=346  \\$acomputer file$bPAL$2rda
=347  \\$avideo file$bAVI$2rda
=508  \\$aכוריאוגרפיה: רינה ברוך. מבוסס על המחזה "המלך אובו" (1896) מאת אלפרד ז'ארי. עריכה מוסיקלית: אלכס קלוד (עפ"י מוסיקה של פרנק זאפה). תלבושות: גלית האן. תאורה: במב.
=508  \\$aChoreography: Rina Baruch. Based on the play "Ubu Roi" (1896) by Alfred Jarry. Music Editing: Alex Claude (according to Frank Zappa). Costumes: Ga

In [357]:
print(create_246(batsheva_records[1091], new_test_record))

=LDR            22        4500
=035  \\$a(IL-BABAL)000500322
=041  \\$aheb
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=093  \\$cBAL-111-6(1990)-2-3-Harvard(D)$dארכיון להקת מחול בת שבע$eBatsheva Dance Company Archive
=100  1\$aTillis, Barry$9lat$elighting designer$g- lighting programs
=100  1\$aTillis, Barry$9lat$elighting designer$g- lighting programs
=110  2\$aלהקת מחול בת שבע$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=110  2\$aלהקת מחול בת שבע$e(יוצרי הארכיון)$9heb
=245  10$aזינה$h[תמונה]
=246  33$iEnglish title:$aKing Ubu$h [videorecording]
=655  \7$aphotographs$2aat$0http://vocab.getty.edu/aat/300046300
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



In [318]:
relators = []
for record in batsheva_records:
    try:
        fields = record.get_fields("100")
        for field in fields:
#             print(record.get_fields('001')[0].value())
#             print(field.value())
            relators.append(field["e"])
            if field["e"] == None:
                print(record.get_fields('001')[0].value())
            
    except:
        continue
list(set(relators))

000425932
000580443
000626691
000626702
000626703
000626817
000628204
000628205
000628208
000639790


[None,
 'choreographer.',
 'choreographer',
 'מלחין',
 'lighting designer',
 'כוריאוגרף',
 'תאורן',
 'במאי']

# Copy MARC Field 256

In [358]:
def copy_256(input_record, new_record):
    """copy MARC Field 256
   """
    record = deepcopy(input_record)
    for field in record.get_fields("256"):
        
        new_record.add_ordered_field(field)
        
    return new_record

In [360]:
print(get_record_by_sysid(batsheva_records, "000639083"))

record index: 1152
=LDR  00000ngm  22      i 4500
=001  000639083
=005  20190219112222.0
=007  cd\cza||||||||
=008  150610t19981998is\|||\\\\\\\\\\\\||heb
=041  \\$aheb
=080  \\$aEStorage BS 002B
=092  \\$a[14]
=100  1\$aSabate, Joaquim$9lat$echoreographer
=245  10$a!טרמולו טרמולו!$h [הקלטת וידאו]
=246  33$aטרמולו טרמולו!$h [הקלטת וידאו]
=246  33$a!Tremolo Tremolo!$h [videorecording]
=246  33$aTremolo Tremolo!$h [videorecording]
=256  \\$aקובץ מחשב - AVI
=264  \1$aתל אביב :$bלהקת מחול בת-שבע,$c[1998].
=264  \4$c©1998.
=300  \\$a2 קבצי מחשב AVI (14 + 12 דק') :$bצבע.
=336  \\$atwo-dimensional moving image$btdi$2rdacontent
=337  \\$avideo$bv$2rdamedia
=338  \\$acomputer file$bd$2rdacarrier
=344  \\$adigital$2rda
=346  \\$acomputer file$bPAL$2rda
=347  \\$avideo file$bAVI$2rda
=508  \\$aכוריאוגרפיה: יואקים סאבטה. יועצת אמנותית: נעמי פרלוב. מוסיקה, קונטרה טנור: יואקים סאבטה. עיצוב סאונד: יותם אגם. תפאורה: יואקים סאבטה. תלבושות: דליה לידר. תאורה: פליס רוס.
=508  \\$aChoreography: Joaquim Sab

In [362]:
print(copy_256(batsheva_records[1152], new_record))

=LDR            22        4500
=080  \\$aBAL 111.6(1990).2.3 זינה_דגון_Harvard_[D]
=245  10$aזינה$h[תמונה]
=256  \\$aקובץ מחשב - AVI
=256  \\$aקובץ מחשב - AVI
=999  \\$aPHOTOGRAPH
=999  \\$aARCHIVE



# Copy MARC Field 260 
in order to coher to the NLI guidelines for field 260, sub field 260$g must be added, and sub field 260$c must contain a normalized date so the NR which fills field 008 will work.

Another problem.
Only 1198 out of 1511 have 260 Field. 
- for those records which don't have 260, the corresponding 008 position should be copy. 
- if positions 07-10 - Date 1 and 11-14 - Date 2  - do not contain any information, than no 260 field will be created


In [365]:
check_field_occurances(batsheva_records,"260")

260 field appears in 1198 out of 1511


In [444]:
no_260 = []
for record in batsheva_records:
    try:
        field = record.get_fields("260")[0]
        print(field)
    except:
        no_260.append((record.get_fields("001")[0].value(), record.get_fields("008")[0].value()))
         

=260  \\$aJerusalem :$bTel Ad,$c[1987?].
=260  \\$a[ישראל] :$b[חמו"ל],$c[2004].
=260  \\$aישראל :$b[חסר מקום ההוצאה],$c[1971].
=260  \\$c1996.
=260  \\$c1996.
=260  \\$c1996.
=260  \\$aישראל :$bמולה את הרמתי,$c1964.
=260  \\$aישראל :$bמולה את הרמתי,$c1964.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1965.
=260  \\$a[ישראל] :$bיעקב אגור,$c1966.
=260  \\$a[ישראל] :$bיעקב אגור,$c1966.
=260  \\$a[ישראל] :$bיעקב אגור,$c1966.
=260  \\$a[ישראל] :$bמולה את הרמתי,$c1966.
=260  \\$a[ישראל] :$bיעקב אגור,$c1967.
=260  \\$a[ישראל] :$bבנרף אלכס,$c1967.
=260  \\$a[ישראל] :$bמולה את הרמתי,$c1968.
=260  \\$a[ישראל] :$bיעקב אגור,$c1968.
=260  \\$a[ישראל] :$bמולה את הרמתי,$c1968.
=260  \\$a[ישראל] :$bמולה את הרמתי,$c1968.
=260  \\$a[ישראל] :$bמולה

In [445]:
import pprint

pprint.pprint(no_260)

[('000500298', '010521s        is      r           heb d'),
 ('000500299', '010524s        is      r           heb d'),
 ('000500300', '010524s        is      r           heb d'),
 ('000500301', '010524s        is      r           heb d'),
 ('000500302', '010524s        is      r           heb d'),
 ('000500303', '010524s        is      r           heb d'),
 ('000500304', '010524s        is      r           heb d'),
 ('000500305', '010524s        is      r           heb d'),
 ('000500306', '010524s        is      r           heb d'),
 ('000500307', '010524s        is      r           heb d'),
 ('000500308', '010524s        is      r           heb d'),
 ('000500309', '010524s        is      r           heb d'),
 ('000500310', '010524s        is      r           heb d'),
 ('000500311', '010528s        is      r           heb d'),
 ('000500312', '010528s        is      r           heb d'),
 ('000500313', '010528s        is      r           heb d'),
 ('000500314', '010528s        is      r

In [420]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [423]:

Authority_instance.df_countries.reset_index().set_index("MARC")[Authority_instance.df_countries.reset_index().set_index("MARC").index.duplicated()]

,מדינת פרסום,מדינת פרסום באנגלית,מדינת פרסום בערבית,הסבר,נרדפות,מדינת פרסום בערבית רמיזות
MARC,,,,,,
,,,,,,
xx#,לא רלוונטי,,صله غير قائمه,כאשר עבור סוג החומר המסויים הזה אין צורך לציין מדינת פרסום,,صله غير قائمه
xxu,"ארה""ב",United States,,,xxu,
,המדינה האסלאמית,,,,,
xxk,הממלכה המאוחדת,United Kingdom,,,xxk,
...,...,...,...,...,...,...
,סומלילנד,,,,,
cc,סין,China,,,cc,
,רפובליקת סהרה הערבית הדמוקרטית,,,,,


In [ ]:
df_

In [446]:
countries = Authority_instance.df_countries.reset_index().set_index("MARC")

countries = countries[~countries.index.duplicated(keep='first')]

In [518]:
vals_260_auth = []
for record in batsheva_records:
    
    try:
        field = record.get_fields("260")[0]
        print(field["a"], field["b"], field["c"])
        print(record.author().replace("heb", "").strip())
        print('\n')
        vals_260_auth.append({
            '001':record.get_fields("001")[0].value(),
            '260$a':field["a"].strip(":").strip(),
            '260$b': field["b"],
            '260$c': field["c"],
            '110/100': record.author().replace("heb", "").strip(),
            '245$c': record.get_fields("245")[0]["c"]
        })
        
    except:
        print(f"no 260 for {record.get_fields('001')[0].value()}")
        date_type = record.get_fields("008")[0].value()[6]
        date1 = record.get_fields("008")[0].value()[7:11]
        date2 = record.get_fields("008")[0].value()[11:14]
        country_code = record.get_fields("008")[0].value()[15:17]
        print(f'date_type: {date_type}, date1={date1}, date2={date2}, country_code={country_code}')
        vals_260_auth.append({
            '001':record.get_fields("001")[0].value(),
            '260$a': "",
            '260$b': "",
            '260$c': "",
            '110/100': "",
            '245$c': "",
            
        })

                            


Jerusalem : Tel Ad, [1987?].
no 260 for 000287253
date_type: s, date1=1987, date2=   , country_code=is
[ישראל] : [חמו"ל], [2004].
איל, שרון


no 260 for 000500298
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500299
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500300
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500301
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500302
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500303
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500304
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500305
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500306
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500307
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500308
date_type: s, date1=    , date2=   , country_code=is
no 260 for 000500309
date_type: s, date1=   

ישראל : להקת מחול בת שבע, [1976].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1975].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, 1976.
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1976].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1976].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, 1976.
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, 1976.
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, [1976].
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, 1978.
להקת מחול בת שבע


None None 1978.
להקת מחול בת שבע


no 260 for 000506650
date_type: s, date1=1978, date2=   , country_code=is
ישראל : להקת מחול בת שבע, 1975.
להקת מחול בת שבע


ישראל : להקת מחול בת שבע, 1983.
להקת מחול בת שבע


יש

ישראל : יורם רובין, 1984.
להקת מחול בת שבע


ישראל : יורם רובין, 1989.
להקת מחול בת שבע


ישראל : יורם רובין, 1984.
להקת מחול בת שבע


ישראל : יורם רובין, 1982.
להקת מחול בת שבע


ישראל : יורם רובין, 1981.
להקת מחול בת שבע


ישראל : יורם רובין, 1981.
להקת מחול בת שבע


ישראל : יורם רובין, 1981.
להקת מחול בת שבע


ישראל : יורם רובין, 1986.
להקת מחול בת שבע


ישראל : יורם רובין, 1987.
להקת מחול בת שבע


ישראל : יורם רובין, 1985.
להקת מחול בת שבע


ישראל : יורם רובין, 1983.
להקת מחול בת שבע


ישראל : יורם רובין, 1984.
להקת מחול בת שבע


ישראל : יורם רובין, 1989.
להקת מחול בת שבע


ישראל : יורם רובין, 1981.
להקת מחול בת שבע


ישראל : יורם רובין, 1981.
להקת מחול בת שבע


ישראל : יורם רובין, 1983.
להקת מחול בת שבע


ישראל : יורם רובין, 1985.
להקת מחול בת שבע


ישראל : יורם רובין, 1985.
להקת מחול בת שבע


ישראל : יורם רובין, 1982.
להקת מחול בת שבע


ישראל : יורם רובין, 1983.
להקת מחול בת שבע


ישראל : יורם רובין, 1982.
להקת מחול בת שבע


ישראל : יורם רובין, 1988.
להקת מחול בת שבע


ישראל : יו

להקת מחול בת שבע


ישראל : דגון, גדי, 1996.
להקת מחול בת שבע


ישראל : דגון, גדי, 2003.
להקת מחול בת שבע


ישראל : דגון, גדי, 1992.
להקת מחול בת שבע


ישראל : דגון, גדי, 1991.
להקת מחול בת שבע


ישראל : דגון, גדי, 2003.
להקת מחול בת שבע


ישראל : דגון, גדי, 2007.
להקת מחול בת שבע


ישראל : דגון, גדי, 2007.
להקת מחול בת שבע


ישראל : דגון, גדי, 1997.
להקת מחול בת שבע


ישראל : דגון, גדי, 1990.
להקת מחול בת שבע


ישראל : דגון, גדי, 1997.
להקת מחול בת שבע


ישראל : דגון, גדי, 2005.
להקת מחול בת שבע


ישראל : דגון, גדי, 1990-.
להקת מחול בת שבע


ישראל : דגון, גדי, 1990.
להקת מחול בת שבע


ישראל : דגון, גדי, 1993.
להקת מחול בת שבע


ישראל : דגון, גדי, 1995.
להקת מחול בת שבע


ישראל : דגון, גדי, [שנה לא ידועה].
להקת מחול בת שבע


ישראל : דגון, גדי, 2008.
להקת מחול בת שבע


ישראל : דגון, גדי, [2002?].
להקת מחול בת שבע


ישראל : דגון, גדי, 1990.
להקת מחול בת שבע


ישראל : בנרף, אלכס, 1965.
להקת מחול בת שבע


ישראל : בנרף אלכס, 1965.
להקת מחול בת שבע


[ישראל] : בנרף אלכס, 1967.
להקת מחול בת שב

להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2011.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2000.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2000.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 1993.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2001.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2002.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2002.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2002.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2004.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2004.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2004.
להקת מחול בת שבע


ישראל : להקת מחול בת-שבע, 2004.
להקת מחול בת שבע


None None 2004.
להקת מחול בת שבע


no 260 for 000706510
date_ty

In [519]:
pd.DataFrame(vals_260_auth).to_excel("batsheva_260_110_100_check1.xlsx")

# Missing date data and re-arrangement of Creators

For the purpose of copyright analysis, it was necessary to add correct creators in cases where the creator was not specific enough for the copyright analysis. 

A manual repair was done, on data from the following fields:
- in 260 field was taken from 2 sources:
    1. the MARCxml export
    2. the Excel spreadsheet table Victoria sent, after manual amemndments she introduced in the data


- in most cases the main creator that is cataloged is Bathsheva, even if its a photograph. In these case, Batsheva was moved to 710 field, and the photographer was extracted from 260 field.

The following part will take the dataframe of the repaired records, and only for those records which appear in the dataframe the 1xx/7xx field will overide the existing 1xx/7xx field from the MARCxml export.




In [521]:
# parse the manual edited spreadsheet into a dataframe.
df_creators_fixed = pd.ExcelFile("./Data/BatSheva/batsheva_260_110_100_repair.xlsx").parse("Sheet1")

In [522]:
df_creators_fixed.head()

,Unnamed: 0,001,260$a,260$b,260$c,110/100,300 from table,245$c,260 from table,100 from file,100$a,100$e,110$a,110$e,710$a,710$e,700$a,700$e
0,57,501508,NaN,NaN,[1988],NaN,1 קובץ מחשב AVI (12 דק') :שחור/לבן.,NaN,"תל אביב :להקת מחול בת-שבע,[1988].",NaN,NaN,NaN,להקת מחול בת-שבע,מחבר,NaN,NaN,NaN,NaN
1,58,501510,NaN,NaN,[1988],NaN,1 קבצי מחשב AVI (57 דק'). :צבע.,NaN,"תל אביב :להקת מחול בת-שבע,[1988].",NaN,NaN,NaN,להקת מחול בת-שבע,מחבר,NaN,NaN,NaN,NaN
2,59,501511,ישראל,NaN,1975,NaN,1 קבצי מחשב AVI & MP4 (43 דק'). :שחור/לבן.,NaN,"תל אביב :להקת מחול בת-שבע,1975.",NaN,NaN,NaN,להקת מחול בת-שבע,מחבר,NaN,NaN,NaN,NaN
3,60,501513,ישראל,NaN,1980,NaN,1 קבצי מחשב AVI & MP4 (30 דק'). :שחור/לבן.,NaN,"תל אביב :להקת מחול בת-שבע,1980.",NaN,NaN,NaN,להקת מחול בת-שבע,מחבר,NaN,NaN,NaN,NaN
4,61,501518,NaN,NaN,[1981],NaN,1 קבצי מחשב MP4 (55 דק') :שחור/לבן.,NaN,"תל-אביב :להקת מחול בת שבע,[1981].",NaN,NaN,NaN,להקת מחול בת-שבע,מחבר,NaN,NaN,NaN,NaN


In [489]:
def copy_and_create_260(input_record, new_record, df_creators_fixed):
    """copy MARC Field 260
   """
    record = deepcopy(input_record)
    
    for field in record.get_fields("260"):
        
        # get date data from 008
        date_type = record.get_fields("008")[0].value()[6]
        date1 = record.get_fields("008")[0].value()[7:11]
        date2 = record.get_fields("008")[0].value()[11:14]
        
        if field is not None:
            
            country_code = record.get_fields("008")[0].value()[15:17]
            country_name = test.loc[country_code, "מדינת פרסום"]
            
            
            if field["a"] is None:
                field["a"] == f'[{country_name}]'
            
            
            

            
            
        else: 
            
        new_field = Field(
            tag = '260',
            indicators = [' ',' '],
            subfields = [
                'a', '',
                'b', 'from journeyman to master /',
                'c', 'Andrew Hunt, David Thomas.'
        ])
        
        
#         new_record.add_ordered_field(field)
        
    return new_record

IndentationError: expected an indented block (<ipython-input-489-7b529ada3613>, line 28)

In [458]:
for record in batsheva_records:
    record.get_fields("008")

is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
i

is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
is ישראל
i

In [532]:
print(get_record_by_sysid(batsheva_records, "000506147"))

record index: 295
=LDR  00000ngm  22      a 4500
=001  000506147
=005  20190401150303.0
=008  060608s1974\\\\is\nnn\\\\\\\\\\\\i|heb\d
=041  \\$aheb
=080  \\$aBAL [Ph_S] 111.6.2.3 נסיך
=110  2\$aלהקת מחול בת שבע$9heb
=245  10$aנסיך האפלה$h [תמונה]
=260  \\$a[ישראל] :$b[מוציא לאור לא ידוע],$c1974.
=300  \\$a5 שקופיות.
=500  \\$aChoreography: John Butler
=500  \\$aMusic: Arnold Schoenberg
=500  \\$aSet: Dani Karavan. costumes: Lydia Pinkus-Gani
=500  \\$aLight: John Bird
=500  \\$aכוריאוגרפיה: ג'ון (ג'והן) באטלר
=500  \\$aמוסיקה: ארנולד שיינברג
=500  \\$aתפאורה: דני קרוון. תלבושות: לידיה פינקוס-גני
=500  \\$aתאורה: ג'ון בריד
=500  \\$aמתנה :אוסף של גיורא מנור
=630  04$aנסיך האפלה (כור' בטלר, ג'ון. 1974)$vתצלומים$x(מחול)
=630  04$aPuppets of Death (Choreographic Work: Butler, john. 1974)$x(Dance)
=610  24$aבת-שבע (להקת מחול)$d1964 - 1989$vתצלומים$x(מחול)$9heb
=650  \4$aDance photography$x(Dance)
=700  1\$aButler, John$9lat$echoreographer$g- photos
=850  \\$aתצלומים
=999  \\$aSLIDE
=999  \

In [397]:
date_type = batsheva_records[1000].get_fields("008")[0].value()[6]
date1 = batsheva_records[1000].get_fields("008")[0].value()[7:11]
date2 = batsheva_records[1000].get_fields("008")[0].value()[11:14]
val_260 = batsheva_records[1000].get_fields('260')[0]

In [398]:
print(f'date_type: {date_type}\ndate1: {date1}\ndate2: {date2}')
print(val_260)

date_type: s
date1: 1995
date2:    
=260  \\$aישראל :$bדגון, גדי,$c1995.


# Add missing 655 MARC field

logic:
- if 300\$c = תצלומים -->   photographs
- if 300\$a contain שקופיות --> photographs
- if 300\$a contains **AVI** --> 
- if 300\$a contains **MP4** --> 
- if 300\$a contains **min.** or **דק'** -->
- if 300\$a contains **תווים**-->
- if 300\$a contains יומן

In [ ]:
3if 300

In [524]:
check_field_occurances(batsheva_records, "655")

655 field appears in 40 out of 1511


In [531]:
for record in batsheva_records:
    try:
        [print(x) for x in record.get_fields("655")]
    except: 
        continue
    try:
        [print(x["a"]) for x in record.get_fields("300")]
    except: 
        continue
        
    

30 min. :
54 דק' :
1 קובץ מחשב AVI (12 דק') :
1 קבצי מחשב AVI (57 דק'). :
1 קבצי מחשב AVI & MP4 (43 דק'). :
1 קבצי מחשב AVI & MP4 (30 דק'). :
1 קבצי מחשב MP4 (55 דק') :
1 קבצי מחשב AVI (7 דק'). :
1 קבצ מחשב AVI (23 דק').
4 קבצי מחשב AVI (43+38+39+39 דק').
2 קבצי מחשב AVI (28+28 דק') :
1 קבצי מחשב AVI (25 דק'). :
1 קבצי מחשב MP4 (46 דק').
6 קבצי מחשב AVI (88+71+62+39+17+62 דק'). :
1 קבץ מחשב AVI (22 דק'). :
2 קבצי מחשב MP4 (15+15 דק'). :
1 קובץ מחשב AVI (44 דק').
1 קבץ מחשב AVI (22 דק'). :
1 קבצי מחשב MPG (62 דק') :
1 קבצי מחשב MP4 (46 דק') :
1 קבצי מחשב AVI (41 דק'). :
1 קבצי מחשב MP4 (31 דק') :
1 קבצי מחשב MP4 (52 דק').
37 קבצי מחשב AVI ( דק'). :
2 קבצי מחשב AVI (7+7 דק') :
1 קובץ מחשב AVI (11 דק') :
2 קבצי מחשב AVI&MP4 (6+5 דק') :
1 קבצי מחשב AVI & MPEG (57 דק') :
19 קבצי מחשב AVI דק'). :
1 קבצי מחשב AVI (10 דק'). :
1 קבצי מחשב AVI & MP4 (62 דק'). :
1 קבצי מחשב MP4 (52 דק') :
3 קבצי מחשב AVI (1+34+21 דק') :
1 קבץ מחשב AVI (18 דק') :
1 קבצי מחשב AVI&MP4 (40 דק'). :
2 קבצי מחשב MP4 (23

# Create Brief records <a class="anchor" id="9"></a>

In [ ]:
with open(f"\Data\BatSheva\BatSheva_brief_output.xml", "wb") as file_output:
    
    writer = XMLWriter(file_output)
    for record in batsheva_records:
        new_record = Record()
        
        

    writer.write(record)
    writer.close()

# working with all records

In [ ]:
from pprint import pprint
for i in range(len(batsheva_records)):
    pprint( batsheva_records[i].as_dict()['fields'])

In [ ]:
for i in range(len(batsheva_records)):
    record = json.loads((batsheva_records[i].as_json()))
    for field in record["fields"]:
        if '080' in field.keys() or 'LKR' in field.keys():
            print(field)

In [ ]:
test = 'file://///ariela02/Mahol/USERS/Archive_dance_library/100_ISRAEL/101_מחול_אמנותי/111_להקות/111.6(1990) להקת בת שבע/111.6(1990).2.4 מופעים - מודעות, פרסומים/POSTERS לחץ כאן על מנת לצפות בקבצים סרוקים (מתוך הארכיון למחול בלבד)'

In [ ]:
test.split("/")

# Working with the spreadsheet

In [ ]:
import pandas as pd

In [ ]:
df = pd.ExcelFile(r'G:\My Drive\National_Library\Python\VC-Dance\DCBS\202108017_בת-שבע בארכיון למחול בבית אריאלה.xlsx').parse('גיליון1')

In [ ]:
df_test = df.copy()
occurances = {}
for index, row in df_test.iterrows():
    if row[" מיון X080"] in occurances.keys():
        continue
    else:
        value = row[" מיון X080"]
# (df.education == '9th').sum()
        
        occurances[value] = (df_test[" מיון X080"]== value).sum()

In [ ]:
df_test.head()

In [ ]:
pd.DataFrame(list(occurances.items()), index=occurances.keys()).to_excel("080_occurances.xlsx")

In [ ]:
df_test = df.copy()
# df.columns.to_series().groupby(level=0).transform('cumcount')
df_test["סימול פרויקט"] = df_test[" מיון X080"].astype(str) + df_test[" מיון X080"].groupby(level=0).transform('cumcount').astype(str)

# df[df[" מיון X080"].duplicated()]

In [ ]:
for index, row in df_test[df_test[" מיון X080"].duplicated()]:


In [ ]:
[name if duplicated == False else name + "another" for duplicated, name in zip(df["סימול פרויקט"].duplicated(), df["סימול פרויקט"])]

In [ ]:

from collections import Counter

Counter(df["סימול פרויקט"])["BABAL-111-6-4"]

In [ ]:
for i in list(zip(df["סימול פרויקט"].duplicated(), df["סימול פרויקט"])):

    duplicated, call_id = i
#     print(duplicated, call_id)
    if duplicated == False:
        counter = 0
        new_call_id = call_id
        print(new_call_id)
    else:
        counter = Counter(df["סימול פרויקט"])
        place_index = 0 
        
        for x in range(0, counter[call_id]):
            
            new_call_id = f'{call_id}_{place_index}'
            place_index+=1
            
            print(new_call_id)
            

In [ ]:
from itertools import chain


out_l = list(chain.from_iterable(zip(df["סימול פרויקט"].to_list(), [f'{x}_{i}' for i, x in enumerate(df["סימול פרויקט"].to_list(), 1)]))) 
out_l

In [ ]:
def transform(l):
    for i, x in enumerate(l, 1):
        yield x
        yield f'{x}_{i}'  # {}_{}'.format(x, i)

In [ ]:
transform(df["סימול פרויקט"].to_list())

In [ ]:

# last_call_number = df.iloc[4, ]
df.iloc[4, 3]

In [ ]:
df.columns

In [ ]:
last_call_number = df.iloc[4, 3]

for index, row in df.iterrows():
    if row["סימול מקורי"] == df
    
    

# create project call numbers

In [ ]:
df["סימול פרויקט"] = df["סימול מקורי"].apply(lambda x: x.replace(".", "-").replace("BAL ", "BABAL-"))

In [ ]:
[print(x) for x in df.columns.values]

In [ ]:
def explode_col_to_new_df(df, col, sep=";", start=0):
    df_explode = (
        df[col]
        .str.split(sep, expand=True)
        .rename(columns=lambda x: col + f"_{start + x + 1}")
    )
    df = pd.concat([df, df_explode], axis=1)
    df = df.fillna("")
    return df
    
    
    

df = df.rename(columns={'LKR היררכיה': 'LKR'})
df = explode_col_to_new_df(df, col='LKR', sep="ANA")

In [ ]:
import re
def get_sysno_from_lkr(value):
    sysno_found = re.findall(r'^(\d*)\s', str(value).lstrip())
    if len(sysno_found) > 0: 
        return sysno_found[0].zfill(9)
    else:
        return ''


In [ ]:
df['parent_sysno'] = df['LKR_2'].apply(get_sysno_from_lkr) 

In [ ]:
df['SysNo'] = df['SysNo'].astype(str).apply(lambda x: x.zfill(9))

In [ ]:
df[['SysNo', 'parent_sysno']]

In [ ]:
df.loc[df[" מיון X080"].str.len().sort_values().index].drop_duplicates(subset='SysNo')


In [ ]:
from treelib import Node, Tree
tree = Tree()


In [ ]:
tree.create_node("BABAL", 0)

In [ ]:
# Creating nodes under root

for i, c in df.iterrows():
    tree.create_node((i, c["SysNo"]), c["SysNo"], parent=0)

In [ ]:
node = tree.get_node("BABAL")

In [ ]:
tree[0]

In [ ]:
parents_not_in_list = []
    
for i, c in df.iterrows():
    print(i, c['SysNo'], f"parent: {c['parent_sysno']}")
    if c['parent_sysno'] not in df['SysNo'].tolist():
        parents_not_in_list.append(c['parent_sysno'])
        continue
    if c['parent_sysno'] != '':
        tree.move_node( c["SysNo"], c["parent_sysno"])
        
        
#         tree.create_node( c[" מיון X080"], c["SysNo"], parent=0)
#     else:
#         tree.( c[" מיון X080"], c["SysNo"], parent=c['parent_sysno'])

        
        # Moving nodes to reflect the parent-child relationship
# for i, c in df.iterrows():
#     if c["parent_sysno"] == c["parent_sysno"]:
#         tree.move_node(c[" מיון X080"], c["parent_sysno"])

In [ ]:
tree.show()

In [ ]:
[tree[node].tag for node in tree.expand_tree(filter = lambda x: x.data == '000500299')]

In [ ]:
df = df.set_index("SysNo")

In [ ]:
# look for the children of a index

children = []

for node in tree.children("000500299"):
    index, sysno = node.tag
    children.append(sysno)

In [ ]:
df.loc[children].head()
    

In [ ]:
print(tree.get_node("000500299"))

tree.children("000500299")

In [ ]:
for node in tree.children(str(input("enter sysno"))):
    print(node.tag[1])

In [ ]:
list(set(list(filter(None, parents_not_in_list))))

In [ ]:
tree.save2file(r'C:\Users\Yaelg\Google Drive\National_Library\Python\VC-Dance\DCBS\BS_tree.txt')
df.to_excel(f'BS_full_table_{dt_now}.xlsx')

# Create 773

In [ ]:
df.loc[3, :]

In [ ]:
df = df.set_index("SysNo")

In [ ]:
df.loc[df.loc["000507375", "parent_sysno"], "סימול פרויקט"]

In [ ]:
df.head()

In [ ]:

for index, row in df.iterrows():
    current_call_number = row["סימול פרויקט"]
    
    if row["parent_sysno"] != '':
        parent_call_number = df.loc[row["parent_sysno"], "סימול פרויקט"]
    
        
        if parent_call_number==row['סימול פרויקט']:
            print("same", current_call_number, parent_call_number)
            
    

In [ ]:

df.to_excel(f"batsheva_check_call_numbers_{dt_now}.xlsx")